In [5]:
import pandas as pd 
#from pandas import DataFrame
csvfile=pd.read_csv('wetherdata.csv')
df_tennis=pd.DataFrame(csvfile)  
df_tennis
 
def entropy(probs):  # Calulate the Entropy of given probability     
    import math     
    return sum( [-prob*math.log(prob, 2) for prob in probs] )      

def entropy_of_list(a_list): # Entropy calculation of list of discrete val ues (YES/NO)     
    from collections import Counter     
    cnt = Counter(x for x in a_list)
    print("No and Yes Classes:",a_list.name,cnt)     
    num_instances = len(a_list)*1.0     
    probs = [x / num_instances for x in cnt.values()]     
    return entropy(probs) # Call Entropy: 

# The initial entropy of the YES/NO attribute for our dataset. 
print(df_tennis['play tennis']) 
total_entropy = entropy_of_list(df_tennis['play tennis']) 
print("Entropy of given PlayTennis Data Set:",total_entropy)

def information_gain(df, split_attribute_name, target_attribute_name, trace=0):     
    print("Information Gain Calculation of ",split_attribute_name)
    '''     
    Takes a DataFrame of attributes,and quantifies the entropy of a target 
    attribute after performing a split along the values of another attribute.   
    '''          
    # Split Data by Possible Vals of Attribute:     
    df_split = df.groupby(split_attribute_name)     
    #print(df_split.groups)     
    for name,group in df_split:         
        print(name)        
        print(group)          
        # Calculate Entropy for Target Attribute, as well as     
        # Proportion of Obs in Each Data-Split     
    nobs = len(df.index) * 1.0     
        #print("NOBS",nobs)     
    df_agg_ent= df_split.agg({target_attribute_name : 
            [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]     
    #print("DFAGGENT",df_agg_ent)     
    df_agg_ent.columns = ['Entropy', 'PropObservations']     
        #if trace: # helps understand what fxn is doing:      
        #print(df_agg_ent)          
        # Calculate Information Gain:     
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )     
    old_entropy = entropy_of_list(df[target_attribute_name]) 
    return old_entropy - new_entropy 
print('Info-gain for Outlook is :'+str( information_gain(df_tennis, 'outlook', 'play tennis')),"\n") 
print('\n Info-gain for Humidity is: ' + str( information_gain(df_tennis, 'humidity', 'play tennis')),"\n") 
print('\n Info-gain for Wind is:' + str( information_gain(df_tennis, 'wind', 'play tennis')),"\n") 
print('\n Info-gain for Temperature is:' + str( information_gain(df_tennis , 'temp','play tennis')),"\n") 

def id3(df, target_attribute_name, attribute_names, default_class=None):
    
       ## Tally target attribute:     
    from collections import Counter     
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO          
    ## First check: Is this split of the dataset homogeneous?     
    if len(cnt) == 1:         
        return next(iter(cnt))          
    ## Second check: Is this split of the dataset empty?     
    # if yes, return a default value     
    elif df.empty or (not attribute_names): return default_class           
    ## Otherwise: This dataset is ready to be divvied up!     
    else:         
        # Get Default Value for next recursive call of this function:         
        default_class = max(cnt.keys()) #[index_of_max] # most common valu e of target attribute in dataset                  
        # Choose Best Attribute to split on:         
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names]         
        index_of_max = gainz.index(max(gainz))          
        best_attr = attribute_names[index_of_max]                  
        # Create an empty tree, to be populated in a moment         
        tree = {best_attr:{}}         
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]   
        # Split dataset         
           # On each split, recursively call this algorithm.         
           # populate the empty tree with subtrees, which         
           # are the result of the recursive call 
        for attr_val, data_subset in df.groupby(best_attr):             
            subtree = id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
            tree[best_attr][attr_val] = subtree        
        return tree 
       
# Get Predictor Names (all but 'class') 
attribute_names = list(df_tennis.columns) 
print("List of Attributes:", attribute_names)  
attribute_names.remove('play tennis')  #Remove the class attribute  
print("Predicting Attributes:", attribute_names) 

# Run Algorithm: 
from pprint import pprint 
tree = id3(df_tennis,'play tennis',attribute_names) 
print("\n\nThe Resultant Decision Tree is :\n") 
pprint(tree)

0      no
1      no
2     yes
3     yes
4     yes
5      no
6     yes
7      no
8     yes
9     yes
10    yes
11    yes
12    yes
13     no
Name: play tennis, dtype: object
No and Yes Classes: play tennis Counter({'yes': 9, 'no': 5})
Entropy of given PlayTennis Data Set: 0.9402859586706309
Information Gain Calculation of  outlook
overcast
     outlook  temp humidity    wind play tennis
2   overcast   hot     high    week         yes
6   overcast  cool   normal    week         yes
11  overcast  mild     high  strong         yes
12  overcast   hot   normal    week         yes
rain
   outlook  temp humidity    wind play tennis
3     rain  mild     high    week         yes
4     rain  cool   normal    week         yes
5     rain  cool   normal  strong          no
9     rain  mild   normal  strong         yes
13    rain  mild     high  strong          no
sunnny
   outlook  temp humidity    wind play tennis
0   sunnny   hot     high    week          no
1   sunnny   hot     high  strong      

In [4]:
def test(attributes,instance,tree):
    attributes=next(iter(tree))
    i = attributes.index(attributes)
    if instance[i] in tree[attributes].keys():
        result = tree[attributes][instance[i]]
        if isinstance(result,dict):
            return test(attributes,instance,result)
        else:
            return result
    else:
        return 'NULL'

In [6]:
        instance=['sunny','hot','high','weak']
        print("*****************")
        print('Testing instance is:',instance)
        result=test(attributes,instance,tree)
        print('The Target value for the testing instance is:')
        print(result)

*****************
Testing instance is: ['sunny', 'hot', 'high', 'weak']


NameError: name 'attributes' is not defined